# Projecto ETL Migracion a PostgreSQL

Proyecto de migracion de datos de una ecommerce a su nueva base de datos en PostgreSQL.

Las librerias utilizadas son:

In [51]:
import pandas as pd
import numpy as np
import os

Utilizamos rutas relativas para el proyecto.

In [52]:
# Definicion de paths
path_ = os.getcwd()
dataset_name = 'jio_mart_items.csv'

Carga del archivo con la rawdata.

contiene las siguientes columnas:

    - category : categoria de la producto
    - subcategory : subcategoria de la producto
    - href : url de la pagina de la producto
    - items : nombre del producto
    - price : precio del producto

In [53]:
# Carga de datos rawdata products
products = pd.read_csv(os.path.join(path_, dataset_name))

En el proceso de normalizacion se generara una tabla de categorias y subcategorias de los productos.

In [55]:
# Create table category
list_category = products['category'].unique()
size_category = len(list_category)
list_id_category = list(range(1,size_category+1))
category = pd.DataFrame({'id_category': list_id_category,
                        'name_category': list_category,
                        })

In [56]:
category

,id_category,name_category
0,1,Groceries
1,2,Home & Kitchen
2,3,Fashion
3,4,Electronics
4,5,Beauty
5,6,Jewellery


In [57]:
# Create table sub-category
list_sub_category = products['sub_category'].unique()
size_sub_category = len(list_sub_category)
list_id_sub_category = list(range(1,size_sub_category+1))
sub_category = pd.DataFrame({'id_sub_category': list_id_sub_category,
                            'name_sub_category': list_sub_category
                            })

In [58]:
sub_category

,id_sub_category,name_sub_category
0,1,Fruits & Vegetables
1,2,Premium Fruits
2,3,Dairy & Bakery
3,4,Staples
4,5,Snacks & Branded Foods
...,...,...
67,68,Wellness
68,69,Fitness
69,70,Ayush
70,71,Treatments


Al analizar la base de datos encontramos que tenia productos repetidos y generamos un algoritmo para eliminar los productos repetidos.

In [59]:
# Trabajar en el update para manejar los productos que se repiten
products_value_counts = products['items'].value_counts().reset_index()
products_value_counts = products_value_counts.rename(columns={'items': 'count', 'index': 'items'})
products = pd.merge(products, products_value_counts, on='items', how='left')
products = products.query('count == 1.0')
size_products = len(products)
list_id_products = list(range(1,size_products+1))
products = products.assign(id_product=list_id_products)
products = pd.merge(products, category, left_on='category', right_on='name_category', how='left')
products = pd.merge(products, sub_category, left_on='sub_category', right_on='name_sub_category', how='left')
products =products.rename(columns={'items': 'name_product'})
products = products[['id_product', 'name_product', 'href', 'price', 'id_category', 'id_sub_category']]

In [60]:
products

,id_product,name_product,href,price,id_category,id_sub_category
0,1,Tender Coconut Cling Wrapped (1 pc) (Approx 90...,https://www.jiomart.com/c/groceries/fruits-veg...,49.0,1,1
1,2,Mosambi 1 kg,https://www.jiomart.com/c/groceries/fruits-veg...,69.0,1,1
2,3,Orange Imported 1 kg,https://www.jiomart.com/c/groceries/fruits-veg...,125.0,1,1
3,4,Banana Robusta 6 pcs (Box) (Approx 800 g - 110...,https://www.jiomart.com/c/groceries/fruits-veg...,44.0,1,1
4,5,Pineapple Queen 1 pc (Approx 700 g - 1200 g),https://www.jiomart.com/c/groceries/fruits-veg...,84.0,1,1
...,...,...,...,...,...,...
130532,130533,Reliance Jewels Ag 99.9 5.6 gm Ganesha Silver ...,https://www.jiomart.com/c/jewellery/fine-jewel...,1669.0,6,72
130533,130534,Reliance Jewels Ag 99.9 4.58 gm Ganesha Silver...,https://www.jiomart.com/c/jewellery/fine-jewel...,1391.0,6,72
130534,130535,Reliance Jewels Ag 99.9 2.31 gm Ganesha Silver...,https://www.jiomart.com/c/jewellery/fine-jewel...,834.0,6,72
130535,130536,Reliance Jewels Ag 99.9 7.76 gm Ganesha Silver...,https://www.jiomart.com/c/jewellery/fine-jewel...,2276.0,6,72
